# 02_entiry-extract
* entity extraction from news description using trasnformer Davlan/xlm-roberta-base-ner-hrl

### install extra library

In [8]:
!pip install transformers torch
!pip install sentencepiece

### read stored data from DataPlatform

In [9]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [10]:
dir_raw_id = 43
dir_process_id = 44

In [11]:
datanode = DataNode()

Init DataNode sucessful!


In [31]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='news.json'))
meta

{'owner': 'thanutchapon',
 'human_size': '50.81 KB',
 'name': 'news.json',
 'description': 'file news.json',
 'directory': 43,
 'path': 'thanutchapon/DSI310-02/rawData/news.json'}

In [32]:
data = json.load(fp)
data

[{'title': 'กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ. ป้องกันไม่ให้เด็กหลุดออกจากระบบ ตั้งเป้า 5 พันคน',
  'detail': 'วานนี้ (6 กันยายน) ที่ชุมชนโรงหวาย เขตสวนหลวง ศานนท์ หวังสร้างบุญ รองผู้ว่าราชการกรุงเทพมหานคร พร้อมด้วย ไกรยส ภัทราวาท ผู้จัดการกองทุนเพื่อความเสมอภาคทางการศึกษา (กสศ.) ปิยะวรรณ จระกา สมาชิกสภากรุงเทพมหานคร (ส.ก.) เขตสวนหลวง ผู้บริหารสำนักการศึกษา และผู้เกี่ยวข้องร่วมลงพื้นที่เพื่อค้นหาเด็กยากจนพิเศษในชุมชนที่มีความเสี่ยงหลุดจากระบบการศึกษา เพื่อล...',
  'DATE': '2022-07-09'},
 {'title': 'เด็กทุกคนต้องได้เรียน! ‘แสนสิริ’ ออกหุ้นกู้ 100 ล้านบาท สนับสนุนกองทุนเพื่อความเสมอภาคทางการศึกษา นำร่องที่ จ.ราชบุรี ตั้งเป้าเด็กหลุดจากระบบการศึกษาเป็น ‘ศูนย์’ ภายใน 3 ปี',
  'detail': 'ในปี 2565 ‘แสนสิริ’ มีพันธกิจใหญ่ โดยมีเป้าหมายในการสนับสนุนกองทุนเพื่อความเสมอภาคทางการศึกษา หรือ กสศ. โดยมีความตั้งใจที่จะ ‘สร้างความเสมอภาค และลดความเหลื่อมล้ำด้านการศึกษา’ ด้วยการประกาศพันธกิจในโครงการ ‘Zero Dropout เด็กทุกคนต้องได้เรียน’ วางแผนพัฒนาคุณภาพการศึกษาไทยอย่างยั่งยืน ระยะยาว 3 ป

In [14]:
#df2 = pd.json_normalize(data)
#df2

### NER (NameEntitiesRegcognition)
* extract entity from news description

In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [16]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/211 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/980 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [17]:
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9993357, 'index': 4, 'word': '▁Wolfgang', 'start': 10, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9990395, 'index': 9, 'word': '▁Berlin', 'start': 33, 'end': 40}]


In [36]:
def post_process(preds):
    data = []
    for i in range(len(preds)): 
        a = preds[i]
        text = a.get('word')
        for j in range(i, len(preds)):
            b = preds[j]
            if b.get('start') == a.get('end'):
                text += b.get('word')
                a = b
        data.append({
            'type': a.get('entity'),
            'text': text
        })
    tmp = []
    for elm in data:
        text = elm.get('text')
        if bool(re.match("[ก-๛]|▁", text)):
            elm.update({'text': text.replace('▁', ' ')})
            tmp.append(elm)
    return tmp

In [46]:
text = data[0]['detail']
text

'วานนี้ (6 กันยายน) ที่ชุมชนโรงหวาย เขตสวนหลวง ศานนท์ หวังสร้างบุญ รองผู้ว่าราชการกรุงเทพมหานคร พร้อมด้วย ไกรยส ภัทราวาท ผู้จัดการกองทุนเพื่อความเสมอภาคทางการศึกษา (กสศ.) ปิยะวรรณ จระกา สมาชิกสภากรุงเทพมหานคร (ส.ก.) เขตสวนหลวง ผู้บริหารสำนักการศึกษา และผู้เกี่ยวข้องร่วมลงพื้นที่เพื่อค้นหาเด็กยากจนพิเศษในชุมชนที่มีความเสี่ยงหลุดจากระบบการศึกษา เพื่อล...'

In [47]:
output = nlp(text)
output

[{'entity': 'B-LOC',
  'score': 0.97698975,
  'index': 8,
  'word': 'ชุมชน',
  'start': 22,
  'end': 27},
 {'entity': 'I-LOC',
  'score': 0.68145823,
  'index': 9,
  'word': 'โรง',
  'start': 27,
  'end': 30},
 {'entity': 'I-LOC',
  'score': 0.9991392,
  'index': 10,
  'word': 'ห',
  'start': 30,
  'end': 31},
 {'entity': 'I-LOC',
  'score': 0.9987367,
  'index': 11,
  'word': 'วาย',
  'start': 31,
  'end': 34},
 {'entity': 'B-LOC',
  'score': 0.9906929,
  'index': 13,
  'word': 'สวน',
  'start': 38,
  'end': 41},
 {'entity': 'I-LOC',
  'score': 0.99921227,
  'index': 14,
  'word': 'หลวง',
  'start': 41,
  'end': 45},
 {'entity': 'B-PER',
  'score': 0.94333845,
  'index': 15,
  'word': '▁',
  'start': 45,
  'end': 46},
 {'entity': 'B-PER',
  'score': 0.92735034,
  'index': 16,
  'word': 'ศ',
  'start': 46,
  'end': 47},
 {'entity': 'B-PER',
  'score': 0.5078819,
  'index': 17,
  'word': 'านนท์',
  'start': 47,
  'end': 52},
 {'entity': 'I-PER',
  'score': 0.7581843,
  'index': 18,
  'w

In [49]:
import re
post_process(output)

[{'type': 'I-LOC', 'text': 'ชุมชนโรงหวาย'},
 {'type': 'I-LOC', 'text': 'โรงหวาย'},
 {'type': 'I-LOC', 'text': 'หวาย'},
 {'type': 'I-LOC', 'text': 'วาย'},
 {'type': 'I-PER', 'text': 'สวนหลวง ศานนท์ หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': 'หลวง ศานนท์ หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': ' ศานนท์ หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': 'ศานนท์ หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': 'านนท์ หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': ' หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': 'หวังสร้างบุญ'},
 {'type': 'I-PER', 'text': 'สร้างบุญ'},
 {'type': 'I-PER', 'text': 'บุญ'},
 {'type': 'B-LOC', 'text': 'กรุงเทพมหานคร'},
 {'type': 'I-PER', 'text': ' ไกรยส ภัทราวาท'},
 {'type': 'I-PER', 'text': 'ไกรยส ภัทราวาท'},
 {'type': 'I-PER', 'text': 'กรยส ภัทราวาท'},
 {'type': 'I-PER', 'text': 'ยส ภัทราวาท'},
 {'type': 'I-PER', 'text': 'ส ภัทราวาท'},
 {'type': 'I-PER', 'text': ' ภัทราวาท'},
 {'type': 'I-PER', 'text': 'ภัทราวาท'},
 {'type': 'I-PER', 'text': 'าวาท'},
 {'type': 'I-PER', 'text': 'ว

## Process news to spo

In [50]:
spo = []
for news in tqdm(data):
    description = news.get('detail') 
    if description == None:
        continue
    ner = nlp(description)
    processed = post_process(ner)
    for kw in processed:
        spo.append({
            'subject': news.get('title', ""),
            'predicate': kw.get('type', ""),
            'object': kw.get('text', "")
        })

  0%|          | 0/20 [00:00<?, ?it/s]

In [51]:
df = pd.DataFrame(spo)
df['predicate'] = df['predicate'].apply(lambda x: x[2:])
df

,subject,predicate,object
0,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,ชุมชนโรงหวาย
1,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,โรงหวาย
2,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,หวาย
3,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,LOC,วาย
4,กทม. ค้นหาเด็กยากจนพิเศษ รับทุนการศึกษาจาก กสศ...,PER,สวนหลวง ศานนท์ หวังสร้างบุญ
...,...,...,...
257,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,ภาคทางการศึกษา
258,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,ทางการศึกษา
259,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,การศึกษา
260,‘โครงการจดหมายลาครู’ เครื่องมือ ‘ค้นหา’ เด็กด้...,ORG,THE STANDARD


### write DataNode to DataPlatform

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="Analysis_News_1", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='news.json')])